In [168]:
import os
import csv
import json
import pandas as pd
import datetime
from google.cloud import storage
from supabase import create_client

In [54]:
# connect to gcloud bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'survai-data-connect.json'
storage_client = storage.Client()
bucket_name = 'survai-dataset'
bucket = storage_client.bucket(bucket_name)

# Connect to database
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
email = os.environ.get("EMAIL")
password = os.environ.get("PASSWORD")
client = create_client(url, key)
user = client.auth.sign_in(email=email, password=password)

In [56]:
table = 'patriot_front_data'

In [173]:
# access temp_videodata_storage folder
blobs = storage_client.list_blobs(bucket_name, prefix='datasets/test_datasets/sparverius_pf/results_08052022/temp_videodata_storage/')
file_list = []

for blob in blobs:

    file = blob.name.split('/')[5]

    if 'od.json' in file or 'stats.json' in file:
        i = file.count('_')

        if i > 1:
            file_name = "_".join(file.split("_", i)[:i])
            if file_name not in file_list:
                file_list.append(file_name)
                client.table(table).insert({'video': file_name}).execute()

        else:
            file_name = file.split('_')[0]
            if file_name not in file_list:
                file_list.append(file_name)
                client.table(table).insert({'video': file_name}).execute()

            
    if 'od.json' in file:
        od = json.loads(blob.download_as_string())

        for k,v in list(od.items()):
            newkey = str(datetime.timedelta(seconds=int(k)))
            od[newkey] = od.pop(k)

        client.table(table).update({'detections_by_second': od}).eq('video', file_name).execute()

    if 'stats.json' in file:
        stats = json.loads(blob.download_as_string())
        for k,v in stats.items():
            client.table(table).update({k: v}).eq('video', file_name).execute()
